In [2]:
import requests
import json
import re
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
from lxml import etree


In [3]:
API_URL = 'https://api.beta.ons.gov.uk/v1/datasets'
RSS_URL = 'https://backfeed.strangecode.com/DxbyswYRHTXxwBFbsk/http://www.ons.gov.uk/releasecalendar%3Frss'
RSS_URL2 = 'https://www.ons.gov.uk/releasecalendar?rss'

In [4]:
def get_publications_and_datasets(link):
    '''
    reads link,
    returns related publications and datasets

    works by looking for href tags at pub/data locations
    '''
    log = requests.get(link)
    soup = BeautifulSoup(log.content, features= 'html')
    
    h3_source = soup.findAll('h3') # Publications under an h3 header
    publications = soup.find_all('a', {'data-gtm-type': 'related-publications'})
    datasets = soup.find_all('a', {'data-gtm-type': 'related-datasets'})

    related_publications = {x.getText(): 'https://www.ons.gov.uk/'+x['href'] for x in publications}
    related_datasets = {x.getText(): 'https://www.ons.gov.uk/'+x['href'] for x in datasets}
    
    return related_publications, related_datasets

def read_publication(
    link, 
    min_text_len = 3, 
    return_main_points = True, return_litags = False, return_ptags = False,
    remove_duplicates = True, preprocess = True):

    '''
    link: url of ONS publication
    min_text_len: minimum number of words in each sentence
    return_ptags: return any text that was in <p> tags
    return_litags: return any text that was in <li> tags
    remove_duplicates: checks and removes duplicate text
    preprocess: 
        removes text with links,
        tralining whitespace
        removes quotation marks
        removes | mark
        removes urls


    NOTE: points in lists (litags) seem to be consistently what
    we want out of this. functionality exists for both list
    and paragraph tags.
    '''

    log = requests.get(link)
    soup = BeautifulSoup(log.content, features = 'html')

    # the reason I am checking for no additional styling attributes
    # is because the ONS data uses the default for the content of interest
    if return_main_points:
        main_points = soup.find(id='main-points')
        main_points = main_points.find_all(lambda tag: tag.name == 'li' and not tag.attrs)
        
    if return_ptags:
        ptags = soup.find_all(lambda tag: tag.name == 'p' and not tag.attrs)
        
    if return_litags:
        litags = soup.find_all(lambda tag: tag.name == 'li' and not tag.attrs)
    
    # concat together
    all_text = []
    if return_main_points:
        [all_text.append(x.get_text()) for x in main_points if len(x.get_text().split()) > min_text_len]
    if return_ptags:
        [all_text.append(x.get_text()) for x in ptags if len(x.get_text().split()) > min_text_len]
    if return_litags:
        [all_text.append(x.get_text()) for x in litags if len(x.get_text().split()) > min_text_len]
    # if return_ptags and return_litags:
    #     all_text = [all_text.appendx.get_text() for x in (ptags + litags) if len(x.get_text().split()) > min_text_len]
    # elif return_ptags and not return_litags:
    #     all_text = [x.get_text() for x in ptags if len(x.get_text().split()) > min_text_len]
    # elif return_litags and not return_ptags:
    #     all_text = [x.get_text() for x in litags if len(x.get_text().split()) > min_text_len]
    # else:
    #     raise ValueError('Did you set both return_ptag and return_litag to False?')
        
    # remove duplicates
    if remove_duplicates:
        all_text = np.unique(all_text)
    
    # preprocess
    if preprocess:
        preprocessed = []
        for string in all_text:

            # remove | because its only present in related_links
            # remove all references to twitter as these are only for following X account
            # remove all self references to ONS, as these are contact details
            if not any(bad_terms in string for bad_terms in [
                '|', 'twitter', 'Twitter', 'Office for National Statistics', 'Totals may not sum due to rounding'
                ]):

                # remove trailing whitespace
                string = string.strip()
                # remove urls
                string = re.sub(r'http\S+', '', string)
                # remove \" symbol 
                string = re.sub('\"', '', string)
                preprocessed.append(string)

        return preprocessed

    return all_text

In [141]:
log = requests.get(RSS_URL2)
soup = BeautifulSoup(log.content, features = 'xml')

articles = soup.findAll('item')
result_list = []
for article in articles:
    title = article.find('title').text
    link = article.find('link').text
    date = article.find('pubDate').text

    publications, datasets = get_publications_and_datasets(link)

    entry = {
        'title': title,
        'link': link,
        'publications': publications,
        'datasets': datasets,
        'date': date
    }

    result_list.append(entry)

In [142]:
[x['title'] for x in result_list]

['Deaths registered weekly in England and Wales, provisional: week ending 4 June 2021',
 'Public Sector Employment, UK: March 2021',
 'Labour market statistics time series: June 2021',
 'Earnings and employment from Pay As You Earn Real Time Information, UK: June 2021',
 'Labour market in the regions of the UK: June 2021',
 'UK labour market: June 2021',
 'Business and individual attitudes towards the future of homeworking, UK: April to May 2021',
 'Wind energy in the UK: June 2021',
 'Coronavirus (COVID-19) Infection Survey, UK: 11 June 2021',
 'Coronavirus and the social impacts on Great Britain: 11 June 2021']

In [143]:
[(num, str(x['publications'])[:20]) for num, x in enumerate(result_list)]

[(0, "{'Deaths registered "),
 (1, "{'Public sector empl"),
 (2, '{}'),
 (3, "{'Earnings and emplo"),
 (4, "{'Labour market in t"),
 (5, "{'Labour market over"),
 (6, "{'Business and indiv"),
 (7, "{'Wind energy in the"),
 (8, "{'Coronavirus (COVID"),
 (9, "{'Coronavirus and th")]

In [144]:
result_list[8]

{'title': 'Coronavirus (COVID-19) Infection Survey, UK: 11 June 2021',
 'link': 'https://www.ons.gov.uk/releases/coronaviruscovid19infectionsurveyuk11june2021',
 'publications': {'Coronavirus (COVID-19) Infection Survey, UK: 11 June 2021': 'https://www.ons.gov.uk//peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/bulletins/coronaviruscovid19infectionsurveypilot/11june2021'},
 'datasets': {'COVID-19 Infection Survey: Scotland': 'https://www.ons.gov.uk//peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/datasets/covid19infectionsurveyscotland',
  'COVID-19 Infection Survey: Wales': 'https://www.ons.gov.uk//peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/datasets/covid19infectionsurveywales',
  'COVID-19 Infection Survey: technical data': 'https://www.ons.gov.uk//peoplepopulationandcommunity/healthandsocialcare/conditionsanddiseases/datasets/covid19infectionsurveytechnicaldata',
  'COVID-19 Infection Survey: Northern Ireland'

In [145]:
read_publication('https://www.ons.gov.uk//peoplepopulationandcommunity/healthandsocialcare/healthcaresystem/bulletins/healthcareexpenditureukhealthaccountsprovisionalestimates/2020', return_main_points = True)

['Growth in government healthcare expenditure is likely to have been driven both by direct costs of the pandemic, such as spending on new testing and tracing services and indirect costs to existing services, such as additional personal protective equipment for frontline healthcare workers.',
 'Growth in non-government healthcare expenditure varied by financing scheme; the coronavirus pandemic reduced household spending on medical services but spending on medical goods increased.',
 'Growth in total healthcare expenditure was driven primarily by a 25% (nominal terms) increase in government healthcare expenditure.',
 'The share of GDP attributed to healthcare rose to around 12.8% in 2020, from 10.2% in 2019.',
 'Total current healthcare expenditure in 2020 is estimated at £269 billion, a nominal-terms increase of 20% on spending in 2019.']

In [146]:
read_publication('https://www.ons.gov.uk//peoplepopulationandcommunity/healthandsocialcare/healthcaresystem/bulletins/healthcareexpenditureukhealthaccountsprovisionalestimates/2020', return_ptags=True)

['As a result of the provisional nature of these estimates and the data used to produce them, users should be aware that our 2020 figures may be subject to substantial revisions one year after release, when the complete set of data sources used to produce the full health accounts are published.',
 'At this stage it is too early to give a detailed breakdown of the growth in expenditure, although later sections of the bulletin provide information from alternative sources that provide evidence of the likely drivers of growth.',
 'COVID-19 has also had a considerable impact on the cost of providing adult social care services, most notably in care homes. Analysis by LaingBussion suggests that increased costs incurred by independent sector providers in England included the purchase of PPE, additional staffing and extensive deep cleaning. They estimate that, up to September 2020, around £3.3 billion of these additional cost pressures were funded by local government.',
 'COVID-19 is likely to 

In [147]:
pd.read_excel('~/Downloads/birthsandinfantmortalitybyethnicity2007to20192.xlsx', sheet_name=3, header = 5, nrows =13)

,Unnamed: 0,Total,Bangladeshi,Indian,Pakistani,Any other Asian background,Black African,Black Caribbean,Any other Black background,Mixed/ multiple,...,Pakistani.2,Any other Asian background.2,Black African.2,Black Caribbean.2,Any other Black background.2,Mixed/ multiple.2,Any Other ethnic group.2,White British.2,White Other.2,Not Stated.2
0,2019,639987,9505,20627,27573,16169,21589,5480,3777,41918,...,5.9,4.7,7.0,5.8,9.7,3.0,3.8,3.3,3.9,5.9
1,2018,656723,9684,20143,26994,15966,21823,5540,3958,41291,...,5.8,4.4,7.6,5.7,7.3,3.9,5.5,3.5,3.5,5.9
2,2017,678728,10089,21157,28135,16664,22734,5718,4113,41381,...,6.6,5.5,7.0,7.3,8.0,4.0,4.5,3.6,4.1,4.8
3,2016,695427,10450,21700,28682,17807,23635,5933,4168,41931,...,6.9,4.5,7.8,7.0,6.9,4.6,5.8,3.7,4.2,6.6
4,2015,696365,9889,21582,28142,17363,23483,5964,4014,40433,...,7.2,4.5,8.2,8.6,7.9,4.6,4.6,3.8,4.3,6.4
5,2014,694610,9764,22217,28645,16251,23176,5990,4375,37450,...,7.5,4.0,7.5,7.0,7.3,4.7,4.5,4.1,4.3,5.9
6,2013,698130,10063,22673,29099,15411,23723,6525,4534,36046,...,8.4,3.9,7.2,8.5,7.4,4.7,5.4,4.0,3.9,7.2
7,2012,729280,10067,23302,29343,17019,24808,7043,4696,37173,...,7.7,4.5,9.2,8.6,7.4,4.3,6.2,4.3,4.4,6.3
8,2011,719582,9847,22725,27948,16079,24457,6943,4751,34643,...,8.8,5.3,9.9,7.0,7.9,5.0,6.4,4.6,4.3,6.8
9,2010,718425,9542,21312,26892,15143,24870,7238,4894,32721,...,7.9,6.6,8.7,8.4,7.3,5.2,6.0,4.4,4.5,6.1


In [148]:
pd.read_excel('~/Downloads/birthsandinfantmortalitybyethnicity2007to20192.xlsx', sheet_name=4, header = 5, nrows =13)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Total,Asian,Black,Mixed/ multiple,Any Other ethnic group,White,...,Black.2,Unnamed: 23,Mixed/ multiple.2,Unnamed: 25,Any Other ethnic group.2,Unnamed: 27,White.2,Unnamed: 29,Not Stated.2,Unnamed: 31
0,2019,"K04000001, J99000001","ENGLAND, WALES AND ELSEWHERE",Country,639987,73874,30846,41918,15523,452248,...,7.1,NaN,3.0,NaN,3.8,NaN,3.4,NaN,5.9,NaN
1,2019,K04000001,ENGLAND AND WALES,Country,639827,73863,30836,41910,15507,452149,...,7.1,NaN,2.9,NaN,3.7,NaN,3.4,NaN,5.7,NaN
2,2019,E92000001,ENGLAND,Country,610140,73051,30475,41082,15186,433689,...,7.1,NaN,2.9,NaN,3.7,NaN,3.3,NaN,5.7,NaN
3,2019,E12000001,NORTH EAST,Region,25737,1116,392,839,525,22763,...,NaN,u,NaN,u,NaN,u,3.7,NaN,NaN,u
4,2019,E12000002,NORTH WEST,Region,79969,9165,2533,3958,1744,57104,...,10.5,NaN,2.5,u,4.0,u,3.5,NaN,4.4,NaN
5,2019,E12000003,YORKSHIRE AND THE HUMBER,Region,58257,7261,1645,2738,1381,43095,...,3.6,u,4.0,u,5.8,u,3.6,NaN,7.0,u
6,2019,E12000004,EAST MIDLANDS,Region,48976,4291,1413,3167,518,37540,...,7.7,u,2.8,u,NaN,u,2.9,NaN,5.8,u
7,2019,E12000005,WEST MIDLANDS,Region,65953,11791,3626,4531,1613,43611,...,6.6,NaN,3.3,u,6.8,u,3.3,NaN,8.9,u
8,2019,E12000006,EAST,Region,67355,5295,2088,4679,937,52689,...,7.6,u,3.0,u,NaN,u,2.8,NaN,6.0,u
9,2019,E12000007,LONDON,Region,117755,24616,15319,12041,6494,57405,...,7.3,NaN,2.8,NaN,3.4,NaN,3.2,NaN,7.4,u


### IGNORE BELOW

In [149]:
log = requests.get('https://api.beta.ons.gov.uk/v1/datasets')
soup = BeautifulSoup(log.content, features = 'xml')

In [150]:
log = requests.get(API_URL)
soup = BeautifulSoup(log.content, features = 'html')

In [151]:
log.json().keys()

dict_keys(['@context', 'count', 'items', 'limit', 'offset', 'total_count'])

In [152]:
[x['title'] for x in log.json()['items']]

['Local authority ageing statistics, based on annual mid-year population estimates',
 'Local authority ageing statistics, population projections for older people',
 'Local authority ageing statistics, older people economic activity',
 'Local authority ageing statistics, net internal migration people aged 65 and over and 85 and over',
 'Local authority ageing statistics, sex ratios for people aged 65 and over and 85 and over',
 'Local authority ageing statistics, household projections for older people',
 'Local authority ageing statistics, projected sex ratios for older people',
 'Deaths registered weekly in England and Wales by age and sex',
 'Deaths registered weekly in England and Wales by region',
 'Faster Indicators - Online Job Advert Estimates',
 'Coronavirus and the latest indicators for the UK economy and society: Shipping indicators',
 "Consumer Prices Index including owner occupiers' housing costs (CPIH)",
 'Population Estimates for UK, England and Wales, Scotland and Norther

In [153]:
[x['links'] for x in log.json()['items']]

[{'editions': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/ageing-population-estimates/editions'},
  'latest_version': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/ageing-population-estimates/editions/time-series/versions/1',
   'id': '1'},
  'self': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/ageing-population-estimates'},
  'taxonomy': {'href': 'https://api.beta.ons.gov.uk/v1/peoplepopulationandcommunity/birthsdeathsandmarriages/ageing'}},
 {'editions': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/ageing-population-projections/editions'},
  'latest_version': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/ageing-population-projections/editions/time-series/versions/1',
   'id': '1'},
  'self': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/ageing-population-projections'},
  'taxonomy': {'href': 'https://api.beta.ons.gov.uk/v1/peoplepopulationandcommunity/birthsdeathsandmarriages/ageing'}},
 {'editions': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/older-

In [154]:
[x['description'] for x in log.json()['items']]

['Indicators included have been derived from the published 2019 mid-year population estimates for the UK, England, Wales, Scotland and Northern Ireland. These are the number of persons and percentage of the population aged 65 years and over, 85 years and over, 0 to 15 years, 16 to 64 years, 16 years to State Pension age, State Pension age and over, median age and the Old Age Dependency Ratio (the number of people of State Pension age per 1000 of those aged 16 years to below State Pension age).\n\nThis dataset has been produced by the Ageing Analysis Team for inclusion in a subnational ageing tool, which was published in July 2020. The tool enables users to compare latest and projected measures of ageing for up to four different areas through selection on a map or from a drop-down menu.',
 'Projected indicators included are derived from the published 2018-based subnational population projections for England, Wales, Scotland and Northern Ireland up to the year 2043. The indicators are th

In [155]:
test = requests.get('https://api.beta.ons.gov.uk/v1/datasets/ageing-population-estimates/editions/time-series/versions/1')

In [156]:
test.json()

{'@context': 'https://cdn.ons.gov.uk/assets/json-ld/context.json',
 'alerts': [],
 'collection_id': 'cmdageingpopulationestimates-c155b6f8c8431bc67905ff252cc3f52f681509aa6dd4e685a77be28d22461b35',
 'dimensions': [{'href': 'https://api.beta.ons.gov.uk/v1/code-lists/calendar-years',
   'id': 'calendar-years',
   'label': 'Time',
   'links': {'code_list': {}, 'options': {}, 'version': {}},
   'name': 'time'},
  {'description': 'Data are available at a local authority, region and country level for England, Wales, Scotland and Northern Ireland.',
   'href': 'https://api.beta.ons.gov.uk/v1/code-lists/administrative-geography',
   'id': 'administrative-geography',
   'label': 'Geography',
   'links': {'code_list': {}, 'options': {}, 'version': {}},
   'name': 'geography'},
  {'description': 'Data are available for All Persons, Males and Females.',
   'href': 'https://api.beta.ons.gov.uk/v1/code-lists/sex',
   'id': 'sex',
   'label': 'Sex',
   'links': {'code_list': {}, 'options': {}, 'versio

In [157]:
log.json()['items'][5].keys()

dict_keys(['contacts', 'description', 'id', 'links', 'methodologies', 'next_release', 'qmi', 'related_datasets', 'release_frequency', 'state', 'title'])

In [158]:
all_keys = []
for item in log.json()['items']:
    for key in item.keys():
        if key not in all_keys:
            all_keys.append(key)

In [159]:
all_keys

['contacts',
 'description',
 'id',
 'keywords',
 'links',
 'next_release',
 'qmi',
 'related_datasets',
 'release_frequency',
 'state',
 'title',
 'methodologies',
 'national_statistic',
 'publications',
 'license',
 'unit_of_measure']

In [160]:
allall_keys = []
for item in log.json()['items']:
    cur_list = list(item.keys())
    allall_keys.append(cur_list)

In [161]:
set.intersection(*[set(list) for list in allall_keys])

{'contacts',
 'description',
 'id',
 'links',
 'next_release',
 'qmi',
 'release_frequency',
 'state',
 'title'}

In [162]:
all_keys

['contacts',
 'description',
 'id',
 'keywords',
 'links',
 'next_release',
 'qmi',
 'related_datasets',
 'release_frequency',
 'state',
 'title',
 'methodologies',
 'national_statistic',
 'publications',
 'license',
 'unit_of_measure']

In [163]:
log.json()['items']

ation-projections-2018-based',
    'title': 'Wales Subnational population projections (local authority): 2018-based'},
   {'href': 'https://www.nisra.gov.uk/publications/2018-based-population-projections-areas-within-northern-ireland',
    'title': '2018-based Population Projections for Areas within Northern Ireland'},
   {'href': 'https://www.ons.gov.uk/peoplepopulationandcommunity/birthsdeathsandmarriages/ageing/articles/subnationalageingtool/2020-07-20',
    'title': 'Subnational ageing tool'}],
  'release_frequency': 'Biennial',
  'state': 'published',
  'title': 'Local authority ageing statistics, projected sex ratios for older people'},
 {'contacts': [{'email': 'health.data@ons.gov.uk',
    'name': 'Sarah Caul',
    'telephone': '+44 (0)1633 456 490'}],
  'description': 'Provisional counts of the number of deaths registered in England and Wales, by age and sex, in the latest weeks for which data are available.',
  'id': 'weekly-deaths-age-sex',
  'keywords': ['Deaths', 'weekly de

In [164]:
log2 = requests.get('https://api.beta.ons.gov.uk/v1/datasets/cpih01')

In [165]:
log2.json()['links']['latest_version']['href']

'https://api.beta.ons.gov.uk/v1/datasets/cpih01/editions/time-series/versions/9'

In [166]:
test = requests.get(log2.json()['links']['latest_version']['href'])

In [167]:
test = requests.get(log2.json()['links']['latest_version']['href'] + '/dimensions')

In [168]:
[x['label'] for x in test.json()['items']]

['Aggregate', 'Geography', 'Time']

In [169]:
test.json()['items'][0]['links']['options']['href']

'https://api.beta.ons.gov.uk/v1/datasets/cpih01/editions/time-series/versions/9/dimensions/aggregate/options'

In [170]:
test2 = requests.get(log2.json()['links']['latest_version']['href'] + '/observations')

In [171]:
test2 = requests.get(test.json()['items'][0]['links']['options']['href'])

In [172]:
requests.get(log2.json()['links']['latest_version']['href'] + '/observations')

<Response [400]>

In [16]:
test = requests.get('https://api.beta.ons.gov.uk/v1/datasets?limit=250')

In [6]:
len(test.json()['items'])

'Local authority ageing statistics, based on annual mid-year population estimates'

In [22]:
all_ids = [test.json()['items'][x]['id'] for x in range(41)]

In [24]:
print(all_ids)

['ageing-population-estimates', 'ageing-population-projections', 'older-people-economic-activity', 'older-people-net-internal-migration', 'older-people-sex-ratios', 'projections-older-people-in-single-households', 'projections-older-people-sex-ratios', 'weekly-deaths-age-sex', 'weekly-deaths-region', 'online-job-advert-estimates', 'faster-indicators-shipping-data', 'cpih01', 'mid-year-pop-est', 'weekly-deaths-health-board', 'weekly-deaths-local-authority', 'regional-gdp-by-year', 'wellbeing-local-authority', 'index-private-housing-rental-prices', 'suicides-in-the-uk', 'childrens-wellbeing', 'gdp-to-four-decimal-places', 'generational-income', 'health-accounts', 'wellbeing-quarterly', 'house-prices-local-authority', 'regional-gdp-by-quarter', 'labour-market', 'tax-benefits-statistics', 'ashe-tables-26', 'ashe-tables-25', 'ashe-tables-27-and-28', 'trade', 'ashe-tables-7-and-8', 'ashe-tables-3', 'ashe-tables-11-and-12', 'ashe-tables-20', 'ashe-tables-9-and-10', 'ashe-table-5', 'life-expec

In [175]:
sav = requests.get('https://api.beta.ons.gov.uk/v1/datasets/cpih01/editions/time-series/versions/9')

In [176]:
[x['id'] for x in test.json()['items']]

['ageing-population-estimates',
 'ageing-population-projections',
 'older-people-economic-activity',
 'older-people-net-internal-migration',
 'older-people-sex-ratios',
 'projections-older-people-in-single-households',
 'projections-older-people-sex-ratios',
 'weekly-deaths-age-sex',
 'weekly-deaths-region',
 'online-job-advert-estimates',
 'faster-indicators-shipping-data',
 'cpih01',
 'mid-year-pop-est',
 'weekly-deaths-health-board',
 'weekly-deaths-local-authority',
 'regional-gdp-by-year',
 'wellbeing-local-authority',
 'index-private-housing-rental-prices',
 'suicides-in-the-uk',
 'childrens-wellbeing']

In [177]:
test.json()['items'][0]['links']

{'editions': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/ageing-population-estimates/editions'},
 'latest_version': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/ageing-population-estimates/editions/time-series/versions/1',
  'id': '1'},
 'self': {'href': 'https://api.beta.ons.gov.uk/v1/datasets/ageing-population-estimates'},
 'taxonomy': {'href': 'https://api.beta.ons.gov.uk/v1/peoplepopulationandcommunity/birthsdeathsandmarriages/ageing'}}

In [178]:
log2.json()['links']['latest_version']['href'] + '/observations'

'https://api.beta.ons.gov.uk/v1/datasets/cpih01/editions/time-series/versions/9/observations'

In [179]:
log2 = requests.get('https://api.beta.ons.gov.uk/v1/datasets/cpih01/editions/time-series/versions/6')

In [180]:
log2.json()

{'@context': 'https://cdn.ons.gov.uk/assets/json-ld/context.json',
 'alerts': [],
 'collection_id': 'cmdcpih-2dd77bce9c8d5d08703147b3d2b27ab78ab927c4f38c9c1f2b4369f3bb165f4a',
 'dimensions': [{'href': 'https://api.beta.ons.gov.uk/v1/code-lists/mmm-yy',
   'id': 'mmm-yy',
   'label': 'Time',
   'links': {'code_list': {}, 'options': {}, 'version': {}},
   'name': 'time'},
  {'href': 'https://api.beta.ons.gov.uk/v1/code-lists/uk-only',
   'id': 'uk-only',
   'label': 'Geography',
   'links': {'code_list': {}, 'options': {}, 'version': {}},
   'name': 'geography'},
  {'href': 'https://api.beta.ons.gov.uk/v1/code-lists/cpih1dim1aggid',
   'id': 'cpih1dim1aggid',
   'label': 'Aggregate',
   'links': {'code_list': {}, 'options': {}, 'version': {}},
   'name': 'aggregate'}],
 'downloads': {'csv': {'href': 'https://download.beta.ons.gov.uk/downloads/datasets/cpih01/editions/time-series/versions/6.csv',
   'size': '5148315'},
  'csvw': {'href': 'https://download.beta.ons.gov.uk/downloads/dataset